In [109]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adeliakhasanova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/adeliakhasanova/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adeliakhasanova/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [217]:
import spacy

In [229]:
import fr_core_news_sm

In [ ]:
#https://dataverse.cirad.fr/dataset.xhtml?persistentId=doi:10.18167/DVN1/MSLEFC

In [482]:
data = pd.read_csv('lest_republicain_clean.csv')

In [513]:
data

,Unnamed: 0,date,summary,text,title,topic,url,relevance
0,0,2021/10/14,Suivez avec nous l'évolution de la pandémie de...,20h47 : 6523 patients hospitalisés et 38 nouve...,En direct. Covid-19 : 6523 patients hospitalis...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...,0.0
1,1,2020/09/21,"Vaccination, taux d’incidence, nombre de patie...","Au jeudi 14 octobre 2021, 393 patients sont ho...",Infographies. Covid-19 : 1 décès supplémentair...,NaN,https://www.estrepublicain.fr/sante/2020/09/21...,1.0
2,2,2021/10/14,"Le laboratoire AstraZeneca, qui propose déjà u...",Le régulateur européen a indiqué jeudi avoir c...,Covid-19. Le régulateur européen évalue Evushe...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...,0.0
3,3,2021/10/14,L'OMS a dévoilé mercredi une nouvelle équipe d...,Ils sont 26 membres proposés par l'OMS pour fa...,Covid-19. Qui sont les nouveaux experts chargé...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...,0.0
4,4,2021/10/14,"Selon une étude américaine publiée mercredi, l...",Les personnes ayant reçu le vaccin contre le C...,Covid-19. Un rappel de Moderna ou Pfizer march...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...,0.0
...,...,...,...,...,...,...,...,...
1269,1363,2021/11/08,"Ouverte depuis le 1er septembre dernier, la ca...",C'est désormais dans un contexte de reprise é...,Infographies. Covid-19 : combien de personnes ...,NaN,https://www.estrepublicain.fr/sante/2021/11/08...,1.0
1270,1364,2021/11/08,Elle frappe déjà les trois quarts du continent...,"Variant delta, efficacité vaccinale en baisse...",Santé. Covid-19 : cinq pistes pour limiter la ...,NaN,https://www.estrepublicain.fr/sante/2021/11/08...,0.0
1271,1365,2021/11/08,"Dans la Meuse, le taux d’incidence au 5 novemb...",« La situation sanitaire en Meuse reste fragi...,Meuse. Covid-19 : le taux d’incidence proche d...,NaN,https://www.estrepublicain.fr/sante/2021/11/08...,1.0
1272,1366,2021/11/03,Il s'agit des départements qui repassent au-de...,Rentrée masquée pour de nombreux écoliers fra...,Epidémie. Covid-19 : voici les départements où...,NaN,https://www.estrepublicain.fr/sante/2021/11/03...,1.0


In [514]:
data_negatives = data[data['relevance']==0]

In [515]:
data_positives = data[data['relevance']==1]

In [516]:
data_positive_split = data_positives.sample(n=int(len(data_positives)*(2/3)))

In [517]:
new_data = data_negatives.append(data_positive_split, ignore_index = True)

In [518]:
new_data

,Unnamed: 0,date,summary,text,title,topic,url,relevance
0,0,2021/10/14,Suivez avec nous l'évolution de la pandémie de...,20h47 : 6523 patients hospitalisés et 38 nouve...,En direct. Covid-19 : 6523 patients hospitalis...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...,0.0
1,2,2021/10/14,"Le laboratoire AstraZeneca, qui propose déjà u...",Le régulateur européen a indiqué jeudi avoir c...,Covid-19. Le régulateur européen évalue Evushe...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...,0.0
2,3,2021/10/14,L'OMS a dévoilé mercredi une nouvelle équipe d...,Ils sont 26 membres proposés par l'OMS pour fa...,Covid-19. Qui sont les nouveaux experts chargé...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...,0.0
3,4,2021/10/14,"Selon une étude américaine publiée mercredi, l...",Les personnes ayant reçu le vaccin contre le C...,Covid-19. Un rappel de Moderna ou Pfizer march...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...,0.0
4,7,2021/10/13,Suivez avec nous l'évolution de la pandémie de...,- Le projet de loi prolongeant jusqu'au 31 jui...,En direct. Covid-19 : ce qu'il faut retenir de...,NaN,https://www.estrepublicain.fr/sante/2021/10/13...,0.0
...,...,...,...,...,...,...,...,...
990,718,2021/07/17,"Le retour du masque sur Nancy, Pont-à-Mousson,...","Le retour du masque sur Nancy, Pont-à-Mousson,...","Meurthe-et-Moselle. Nancy, Pont-à-Mousson, bas...",NaN,https://www.estrepublicain.fr/sante/2021/07/17...,1.0
991,106,2021/09/22,"Employée au vaccinodrome de Bordeaux, la jeune...","Une jeune femme de 19 ans, employée du plus gr...",Covid-19. Elle avait fait une vingtaine de fau...,NaN,https://www.estrepublicain.fr/sante/2021/09/22...,1.0
992,924,2021/06/19,La CCI du Doubs et l’Union des Métiers et des ...,La réouverture était annoncée délicate pour le...,Besançon. Hôtellerie et restauration : quatre ...,NaN,https://www.estrepublicain.fr/sante/2021/06/19...,1.0
993,1352,2021/11/10,"Une dose de rappel pour qui, pour quand? Quell...","Alors que l'épidémie de Covid-19 rebondit, Em...",Covid-19. Vaccination : tout savoir sur la dos...,NaN,https://www.estrepublicain.fr/sante/2021/11/10...,1.0


In [538]:
#removing irrelevant columns
data_short = data[['text','relevance']]

In [548]:
data_short

,text,relevance
0,20h47 : 6523 patients hospitalisés et 38 nouve...,0.0
1,"Au jeudi 14 octobre 2021, 393 patients sont ho...",1.0
2,Le régulateur européen a indiqué jeudi avoir c...,0.0
3,Ils sont 26 membres proposés par l'OMS pour fa...,0.0
4,Les personnes ayant reçu le vaccin contre le C...,0.0
...,...,...
1269,C'est désormais dans un contexte de reprise é...,1.0
1270,"Variant delta, efficacité vaccinale en baisse...",0.0
1271,« La situation sanitaire en Meuse reste fragi...,1.0
1272,Rentrée masquée pour de nombreux écoliers fra...,1.0


In [549]:
data_short[data_short['relevance']==0]

,text,relevance
0,20h47 : 6523 patients hospitalisés et 38 nouve...,0.0
2,Le régulateur européen a indiqué jeudi avoir c...,0.0
3,Ils sont 26 membres proposés par l'OMS pour fa...,0.0
4,Les personnes ayant reçu le vaccin contre le C...,0.0
7,- Le projet de loi prolongeant jusqu'au 31 jui...,0.0
...,...,...
1253,"- Gérard Larcher, président du Sénat, refuse ...",0.0
1255,16h24 : Le régulateur européen approuve deux ...,0.0
1257,"23h30 : C'est la fin de ce live, merci de l'a...",0.0
1270,"Variant delta, efficacité vaccinale en baisse...",0.0


In [550]:
def preprocess(text):
    """this function deletes extra characters,
    converts strings to lowercase, and removes punctuation"""
    
    text = text.lower() 
    text = text.strip()  
    text = re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

In [551]:
def stopword(string):
    """this function removes french stopwords"""
    a = [i for i in string.split() if i not in stopwords.words('french')]
    return ' '.join(a)

In [552]:
# Tokenize the sentence
def lemmatizer(string):
    """this function applies french lemmatizer
    and tokenizes the text at the same time"""
    nlp = spacy.load("fr_core_news_sm")
    return ' '.join([token.lemma_ for token in nlp(string)])

In [553]:
def final_transformation(string): 
    """this function puts all the previous steps together"""
    return lemmatizer(stopword(preprocess(string)))

In [547]:
data_short

,text,relevance
0,20h47 : 6523 patients hospitalisés et 38 nouve...,0.0
1,"Au jeudi 14 octobre 2021, 393 patients sont ho...",1.0
2,Le régulateur européen a indiqué jeudi avoir c...,0.0
3,Ils sont 26 membres proposés par l'OMS pour fa...,0.0
4,Les personnes ayant reçu le vaccin contre le C...,0.0
...,...,...
1269,C'est désormais dans un contexte de reprise é...,1.0
1270,"Variant delta, efficacité vaccinale en baisse...",0.0
1271,« La situation sanitaire en Meuse reste fragi...,1.0
1272,Rentrée masquée pour de nombreux écoliers fra...,1.0


In [526]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
data_short['clean_text'] = data_short['text'].apply(final_transformation)

In [498]:
df_X, df_y = data_short['clean_text'], data_short['relevance'] 

In [528]:
x_train, x_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=0)

In [529]:
x_train

363    crise sanitaire question réponse sommaire doss...
440    réponse oui tout rendez premier deuxième dose ...
669    tester alerter isoler credo gouvernement lutte...
263    masqu nouveau obligatoire confinemer retour té...
17     installer lespac montrichard depuis début lann...
                             ...                        
835    benjamin an étudier lensmm cela avoir très fac...
192    printemps nombreux pays dont france vaccin bri...
629    alain seid président chambre commerce dindustr...
559    contrôle sanitaire personne devoir devenir pri...
684    plusieurs millier personne manifester france m...
Name: clean_text, Length: 598, dtype: object

In [530]:
tf_idf = Pipeline([('cv',CountVectorizer()), ('tfidf_transformer',TfidfTransformer(smooth_idf=True,use_idf=True))])


x_train_CV  = tf_idf.fit_transform(x_train)
x_test_CV = tf_idf.transform(x_test)

In [531]:
mnb = MultinomialNB()
y_train=y_train.astype('int')
y_train

363    0
440    1
669    1
263    0
17     0
      ..
835    1
192    0
629    1
559    1
684    1
Name: relevance, Length: 598, dtype: int64

In [532]:
mnb.fit(x_train_CV,y_train)

MultinomialNB()

In [533]:
predictions = mnb.predict(x_test_CV)

In [534]:
from sklearn.metrics import confusion_matrix
confusion_matrix(predictions, y_test)

array([[133,  75],
       [  3,  46]])

In [535]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
            TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
            FP += 1
        if y_actual[i]==y_hat[i]==0:
            TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
            FN += 1

    return('true positive:', TP, 'false positive:', FP, 'true negative:', TN, 'false negative', FN)

In [536]:
perf_measure(list(y_test),list(predictions))

('true positive:',
 46,
 'false positive:',
 3,
 'true negative:',
 133,
 'false negative',
 75)

In [508]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, predictions))

Accuracy: 0.6964980544747081


In [509]:
print("Precision:",metrics.precision_score(y_test, predictions))

Precision: 0.9387755102040817


In [510]:
print("Recall:",metrics.recall_score(y_test, predictions))

Recall: 0.38016528925619836


In [511]:
print("F1:",metrics.f1_score(y_test, predictions))

F1: 0.5411764705882354


In [370]:
from sklearn.linear_model import LogisticRegression

In [375]:
y_train

238     0
115     1
333     0
901     1
34      1
       ..
360     1
709     1
439     1
174     1
1146    1
Name: relevance, Length: 1019, dtype: int64

In [ ]:
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
model=scikit_log_reg.fit(X_train,y_train)